In [ ]:
# CONNECT GOOGLE DRIVE WITH GOOGLE COLAB
# from google.colab import drive
# drive.mount("/content/drive")

# IMPORT LIBRARY

In [ ]:
# IMPORT LIBRARY

# !pip install pdfplumber
import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import BigramCollocationFinder
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import*
nltk.download('punkt')
nltk.download('stopwords')
# nltk.download('wordnet')
import pdfminer
from pdfminer.high_level import extract_text
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

# IMPORT CV

In [ ]:
# IMPORT CV

# Read PDF file and extract text
def read_pdf(file_path):
    text = extract_text(file_path)
    # print(text)
    return text
resume_path = r"data\CV_Mukhamad_Ikhsanudin.pdf"
text = read_pdf(resume_path)
print(text)

# Additional stopwords
f2=open(r"data\stopwords.txt",'r', errors = 'ignore')
text2=f2.read()
stopwords_additional = word_tokenize(text2.replace("\n", " "))

# CV PROCESS FUNCTION

In [ ]:
def nlp(text):
    word_sent = word_tokenize(text.lower().replace("\n",""))
    _stopwords = set(stopwords.words('english', 'indonesian') + list(punctuation)+list("●")+list('–')+list('’')+stopwords_additional)
    word_sent=[word for word in word_sent if word not in _stopwords]
    lemmatizer = WordNetLemmatizer()
    NLP_Processed_CV = [lemmatizer.lemmatize(word) for word in word_tokenize(" ".join(word_sent))]
#     return " ".join(NLP_Processed_CV)
    return NLP_Processed_CV

In [ ]:
def get_key_word(x):
    finder = BigramCollocationFinder.from_words(x)
    keyword = []
    keywords_CV = []
    for i in sorted(finder.ngram_fd.items()):
    # if a double word appears more than once, then print it out.
        if i[1] > 1:
            print(i)
            keyword.append(i[0])
        else:
            pass
    # print("************************")
    for j in keyword:
    #     print(" ".join(j))
        keywords_CV.append(" ".join(j))
    return keywords_CV

In [ ]:
NLP_Processed_CV = nlp(text)
NLP_Processed_CV
key_word = get_key_word(NLP_Processed_CV)
key_word

# IMPORT JOB DATA

In [ ]:
#Import scraped data
df = pd.read_excel(r"data\data website indeed Jakarta (30 May 2023) last.xlsx", "Part-Time")
index = []
for i in range (0, len(df)):
    index.append(i)
df.insert(0,'JobID', index)
df['All'] = df['Posisi'] + ' ' + df['Perusahaan '] + ' ' + df['Lokasi'] + ' ' + df['Tipe Pekerjaan'] + ' ' + df['Kualifikasi'] + ' ' + df['Deskripsi Pekerjaan ']
df.rename(columns = {'Perusahaan ':'Perusahaan', 'Deskripsi Pekerjaan ': 'Deskripsi Pekerjaan'}, inplace = True)
df.shape
df.info()
df.head(2)

# TRAINING JOB DATA

In [ ]:
# TRAINING DATA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
tfidf_jobid = tfidf_vectorizer.fit_transform(df['All'].apply(lambda x: np.str_(x)))
# print(tfidf_jobid)
user_tfidf = tfidf_vectorizer.transform(df['All'].apply(lambda x: np.str_(x)))
# print(user_tfidf)

cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)
outputJob = list(cos_similarity_tfidf)
outputJob

In [ ]:
df2 = pd.DataFrame()

# append columns to an empty DataFrame
df2['Posisi'] = ["I"]
df2['Deskripsi Pekerjaan'] = ["I"]
df2['Perusahaan'] = ["I"]

# Compare with the key words from CV only
df2['All'] = " ".join(key_word)
df2.head()
# Compare with the entire CV
# df2['All'] = " ".join(NLP_Processed_CV)
# df2

# RECOMMENDATION FUNCTION

In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['JobID',  'Posisi', 'Perusahaan', 'Deskripsi Pekerjaan','Score'])
    count = 0
    for i in top:
#         recommendation.at[count, 'ApplicantID'] = u
        recommendation.at[count, 'JobID'] = df.index[i]
        recommendation.at[count, 'Posisi'] = df['Posisi'][i]
        recommendation.at[count, 'Perusahaan'] = df['Perusahaan'][i]
#         recommendation.at[count, 'location'] = df['location'][i]
        recommendation.at[count, 'Deskripsi Pekerjaan'] = df['Deskripsi Pekerjaan'][i]
        recommendation.at[count, 'Score'] =  scores[count]
        count += 1
    return recommendation

def get_top(outputData):
    if outputData.__class__ == tuple:
        top = outputData[1][0][0:]
        list_scores = outputData[0][0][0:]
    else:
        top = sorted(range(len(outputData)), key=lambda i: outputData[i], reverse=True)[:100]
        list_scores = [outputData[i][0][0] for i in top]
    return top, list_scores

In [ ]:
# TFIDF
def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))

    # TF-IDF Scraped data
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data.apply(lambda x: np.str_(x)))

    # TF-IDF CV
    user_tfidf = tfidf_vectorizer.transform(cv.apply(lambda x: np.str_(x)))

    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)

    outputTF = list(cos_similarity_tfidf)
    return outputTF

In [ ]:
# COUNTVECTOR
def count_vectorize(scraped_data, cv):
    count_vectorizer = CountVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
    # CountV the scraped data
    count_jobid = count_vectorizer.fit_transform(scraped_data.apply(lambda x: np.str_(x)))

    # CountV the cv
    user_count = count_vectorizer.transform(cv.apply(lambda x: np.str_(x)))
    
    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
    
    output3 = list(cos_similarity_countv)
    return output3

In [ ]:
# KNN
def KNN(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8', stop_words=stopwords.words('english', 'indonesian'))
    # n_neighbors = 100
    KNN = NearestNeighbors(n_neighbors=10)
    KNN.fit(tfidf_vectorizer.fit_transform(scraped_data.apply(lambda x: np.str_(x))))
#     NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv), return_distance=True)
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv.apply(lambda x: np.str_(x))))
    # knn = get_recommendation(top, df, index_score)
    return NNs

# CV TEST WITH TF-IDF, COUNT VECTORIZER, KNN 

In [ ]:
outputTF = TFIDF(df['All'], df2['All'])
outputTF

In [ ]:
outputCV = count_vectorize(df['All'], df2['All'])
outputCV

In [ ]:
# outputKNN = KNN(df['All'], df2['All'])
# outputKNN
outputKNN = KNN(df['All'], df2['All'])
outputKNN

In [ ]:
top, list_score = get_top(outputTF)
finalTF = get_recommendation(top, df, list_score)
finalTF

In [ ]:
top, list_score = get_top(outputCV)
finalCV = get_recommendation(top, df, list_score)
finalCV

In [ ]:
# top = outputKNN[1][0][0:]
# list_score = outputKNN[0][0][0:]
top, list_score = get_top(outputKNN)
finalKNN = get_recommendation(top, df, list_score)
finalKNN

# MERGE SCORE

In [ ]:
merge1 = finalKNN[['JobID','Posisi', 'Score']].merge(finalTF[['JobID','Score']], on= "JobID")
final = merge1.merge(finalCV[['JobID','Score']], on = "JobID")
final = final.rename(columns={"Score_x": "KNN", "Score_y": "TF-IDF","Score": "CV"})
final.head()

In [ ]:
# SCALE
slr = MinMaxScaler()
final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])

# Multiply by weights
final['KNN'] = (1-final['KNN'])/3
final['TF-IDF'] = final['TF-IDF']/3
final['CV'] = final['CV']/3
final['Final'] = final['KNN']+final['TF-IDF']+final['CV']
final.sort_values(by="Final", ascending=False)

In [ ]:
final_all = df.merge(final, on="JobID")
final_job = final_all.sort_values(by="Final", ascending=False).copy()
final_job

# EXPORT TO PICKLE

In [ ]:
import pickle
pickle.dump(finalTF, open('artifacts/TF.pkl', 'wb'))
pickle.dump(finalCV, open('artifacts/cv.pkl', 'wb'))
pickle.dump(finalKNN, open('artifacts/knn.pkl', 'wb'))
pickle.dump(final, open('artifacts/final.pkl', 'wb'))
pickle.dump(final_job, open('artifacts/final_job.pkl', 'wb'))